In [1]:
from pathlib import Path
from netCDF4 import Dataset as NetCDFFile 
import matplotlib.pyplot as plt
import numpy as np
# from mpl_toolkits.basemap import Basemap
import cartopy.crs as ccrs
import cartopy.feature as cfeature
# %matplotlib inline
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy.feature import ShapelyFeature
import cartopy.io.shapereader as shpreader
from mpl_toolkits.axes_grid1 import AxesGrid, ImageGrid
import pandas as pd 
import geopandas as gpd
import pygeos
import seaborn as sns
import xarray as xr
import rioxarray as rio
from matplotlib.colors import LinearSegmentedColormap
import datetime
from datetime import timedelta
from scipy.spatial.distance import cdist
from math import dist
from scipy.stats import ttest_ind
from geopy import distance
from scipy import stats
from matplotlib.gridspec import GridSpec
from pylr2 import regress2
from matplotlib.dates import (AutoDateLocator, YearLocator, MonthLocator,
                              DayLocator, WeekdayLocator, HourLocator,
                              MinuteLocator, SecondLocator, MicrosecondLocator,
                              RRuleLocator, rrulewrapper, MONTHLY,
                              MO, TU, WE, TH, FR, SA, SU, DateFormatter,
                              AutoDateFormatter, ConciseDateFormatter)
from matplotlib.ticker import MaxNLocator,FixedLocator
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
from matplotlib.colors import Normalize
from scipy.stats import gaussian_kde

# import shutil

/Users/weitao/anaconda3/lib/python3.10/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


# Get hour Pandora

In [2]:
lisPD = []
# total VCD
for path in Path('./fuh5/').rglob('*fuh*.txt'):
    lisPD.append(path)
print(len(lisPD))
lisPD[:5]

19


[PosixPath('fuh5/Pandora176s1_Tsukuba-NIES_L2_rfuh5p1-8.txt'),
 PosixPath('fuh5/Pandora194s1_Tokyo-TMU_L2_rfuh5p1-8.txt'),
 PosixPath('fuh5/Pandora193s1_Tsukuba_L2_rfuh5p1-8.txt'),
 PosixPath('fuh5/Pandora163s1_Tsukuba-NIES-West_L2_rfuh5p1-8.txt'),
 PosixPath('fuh5/Pandora197s1_Nagoya_L2_rfuh5p1-8.txt')]

In [4]:
pandora = pd.DataFrame(columns = ['Time', 'L1QA', 'L2fitQA', 'L2HCHOQA', 'VCD(moles/m2)', 'Status',
       'Location', 'Country', 'Lat', 'Lon', 'Altitude', 'Year', 'Month', 'Day',
       'Hour', 'Minute', 'Datetime', 'Localtime'])

for i in lisPD:
    cols = list(range(1,79))
    # Read txt to df 22+55
    PD = pd.read_csv(i,skiprows=78, encoding='unicode_escape',delimiter=' '
    #                  ,header=0
                     ,names=cols
                    )
    PD = PD.iloc[:, [0,35,38,41,48]]

    cols2 = ['Time','L1QA','L2fitQA', 'L2HCHOQA','VCD(moles/m2)']
    PD.columns = cols2
    # Read Attributes
    file = open(i, encoding='unicode_escape')
    content = file.readlines()
    status = content[4][43:-1]
    location = content[13][21:-1]
    country = content[14][21:-1]
    lat = float(content[15][25:-1])
    lon = float(content[16][26:-1])
    altitude = float(content[17][23:-1])
    
    timezone = int(round(lon/15))
    
    PD['Status'] = status
    PD['Location'] = location
    PD['Country'] = country
    PD['Lat'] = lat
    PD['Lon'] = lon
    PD['Altitude'] = altitude
    
    yearLis = []
    monthLis = []
    dayLis = []
    hourLis = []
    minuteLis = []
    datetimeLis = []
    for j in PD['Time']:
        yearLis.append(j[:4])
        monthLis.append(j[4:6])
        dayLis.append(j[6:8])
        hourLis.append(j[9:11])
        minuteLis.append(j[11:13])
        datetime_obj = datetime.datetime(int(j[:4]), int(j[4:6]), int(j[6:8]), int(j[9:11]), int(j[11:13]))
        datetimeLis.append(datetime_obj)
    PD['Year'] = yearLis
    PD['Month'] = monthLis
    PD['Day'] = dayLis
    PD['Hour'] = hourLis
    PD['Minute'] = minuteLis
    PD['Datetime'] = datetimeLis
    PD['Localtime'] = PD['Datetime'] + pd.Timedelta(hours=timezone)
    pandora = pd.concat([pandora,PD])
pandora

,Time,L1QA,L2fitQA,L2HCHOQA,VCD(moles/m2),Status,Location,Country,Lat,Lon,Altitude,Year,Month,Day,Hour,Minute,Datetime,Localtime
0,20210805T011209.5Z,0,0,10,0.000317,unvalidated,Tsukuba-NIES,Japan,36.0513,140.121,45.0,2021,08,05,01,12,2021-08-05 01:12:00,2021-08-05 10:12:00
1,20210805T011657.0Z,0,0,10,0.000340,unvalidated,Tsukuba-NIES,Japan,36.0513,140.121,45.0,2021,08,05,01,16,2021-08-05 01:16:00,2021-08-05 10:16:00
2,20210805T012045.7Z,0,0,10,0.000342,unvalidated,Tsukuba-NIES,Japan,36.0513,140.121,45.0,2021,08,05,01,20,2021-08-05 01:20:00,2021-08-05 10:20:00
3,20210805T012348.6Z,0,0,10,0.000339,unvalidated,Tsukuba-NIES,Japan,36.0513,140.121,45.0,2021,08,05,01,23,2021-08-05 01:23:00,2021-08-05 10:23:00
4,20210805T014241.8Z,0,0,10,0.000411,unvalidated,Tsukuba-NIES,Japan,36.0513,140.121,45.0,2021,08,05,01,42,2021-08-05 01:42:00,2021-08-05 10:42:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151813,20240921T082508.6Z,11,12,12,0.000192,unvalidated,Seoul-SNU,"Korea, Republic of",37.4580,126.951,116.0,2024,09,21,08,25,2024-09-21 08:25:00,2024-09-21 16:25:00
151814,20240921T083248.2Z,10,12,12,-0.000004,unvalidated,Seoul-SNU,"Korea, Republic of",37.4580,126.951,116.0,2024,09,21,08,32,2024-09-21 08:32:00,2024-09-21 16:32:00
151815,20240921T084156.4Z,12,12,12,0.000071,unvalidated,Seoul-SNU,"Korea, Republic of",37.4580,126.951,116.0,2024,09,21,08,41,2024-09-21 08:41:00,2024-09-21 16:41:00
151816,20240921T085346.8Z,10,12,12,0.000115,unvalidated,Seoul-SNU,"Korea, Republic of",37.4580,126.951,116.0,2024,09,21,08,53,2024-09-21 08:53:00,2024-09-21 16:53:00


In [5]:
pandora = pandora[pandora['Year'].isin(['2021','2022','2023'])]
print(len(pandora))
pandora.head()

1353601


,Time,L1QA,L2fitQA,L2HCHOQA,VCD(moles/m2),Status,Location,Country,Lat,Lon,Altitude,Year,Month,Day,Hour,Minute,Datetime,Localtime
0,20210805T011209.5Z,0,0,10,0.000317,unvalidated,Tsukuba-NIES,Japan,36.0513,140.121,45.0,2021,08,05,01,12,2021-08-05 01:12:00,2021-08-05 10:12:00
1,20210805T011657.0Z,0,0,10,0.000340,unvalidated,Tsukuba-NIES,Japan,36.0513,140.121,45.0,2021,08,05,01,16,2021-08-05 01:16:00,2021-08-05 10:16:00
2,20210805T012045.7Z,0,0,10,0.000342,unvalidated,Tsukuba-NIES,Japan,36.0513,140.121,45.0,2021,08,05,01,20,2021-08-05 01:20:00,2021-08-05 10:20:00
3,20210805T012348.6Z,0,0,10,0.000339,unvalidated,Tsukuba-NIES,Japan,36.0513,140.121,45.0,2021,08,05,01,23,2021-08-05 01:23:00,2021-08-05 10:23:00
4,20210805T014241.8Z,0,0,10,0.000411,unvalidated,Tsukuba-NIES,Japan,36.0513,140.121,45.0,2021,08,05,01,42,2021-08-05 01:42:00,2021-08-05 10:42:00


In [6]:
def hour_rounder(t):
    # Rounds to nearest hour by adding a timedelta hour if minute >= 30
    return (t.replace(second=0, microsecond=0, minute=0, hour=t.hour)
               +timedelta(hours=t.minute//30))
pandora['Datetime1']=pandora['Datetime'].map(hour_rounder)
pandora.head()

/var/folders/4y/knznw84s4p5d2d77tb6wyq6h0000gn/T/ipykernel_84130/33972301.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pandora['Datetime1']=pandora['Datetime'].map(hour_rounder)


,Time,L1QA,L2fitQA,L2HCHOQA,VCD(moles/m2),Status,Location,Country,Lat,Lon,Altitude,Year,Month,Day,Hour,Minute,Datetime,Localtime,Datetime1
0,20210805T011209.5Z,0,0,10,0.000317,unvalidated,Tsukuba-NIES,Japan,36.0513,140.121,45.0,2021,08,05,01,12,2021-08-05 01:12:00,2021-08-05 10:12:00,2021-08-05 01:00:00
1,20210805T011657.0Z,0,0,10,0.000340,unvalidated,Tsukuba-NIES,Japan,36.0513,140.121,45.0,2021,08,05,01,16,2021-08-05 01:16:00,2021-08-05 10:16:00,2021-08-05 01:00:00
2,20210805T012045.7Z,0,0,10,0.000342,unvalidated,Tsukuba-NIES,Japan,36.0513,140.121,45.0,2021,08,05,01,20,2021-08-05 01:20:00,2021-08-05 10:20:00,2021-08-05 01:00:00
3,20210805T012348.6Z,0,0,10,0.000339,unvalidated,Tsukuba-NIES,Japan,36.0513,140.121,45.0,2021,08,05,01,23,2021-08-05 01:23:00,2021-08-05 10:23:00,2021-08-05 01:00:00
4,20210805T014241.8Z,0,0,10,0.000411,unvalidated,Tsukuba-NIES,Japan,36.0513,140.121,45.0,2021,08,05,01,42,2021-08-05 01:42:00,2021-08-05 10:42:00,2021-08-05 02:00:00


In [10]:
pandora2 = pandora
print(len(pandora2[pandora2['Location']=='Agam']))
pandora2 = pandora2[pandora2['L1QA'].isin([0,10])]
print(len(pandora2[pandora2['Location']=='Agam']))

pandora2 = pandora2[pandora2['L2fitQA'].isin([0,10])]
print(len(pandora2[pandora2['Location']=='Agam']))

# pandora2 = pandora2[~pandora2['L2HCHOQA'].isin([20,21,22])]
pandora2 = pandora2[pandora2['L2HCHOQA'].isin([0,1,10,11])]
print(len(pandora2[pandora2['Location']=='Agam']))

pandora2

40678
22081
22
21


,Time,L1QA,L2fitQA,L2HCHOQA,VCD(moles/m2),Status,Location,Country,Lat,Lon,Altitude,Year,Month,Day,Hour,Minute,Datetime,Localtime,Datetime1
0,20210805T011209.5Z,0,0,10,0.000317,unvalidated,Tsukuba-NIES,Japan,36.0513,140.121,45.0,2021,08,05,01,12,2021-08-05 01:12:00,2021-08-05 10:12:00,2021-08-05 01:00:00
1,20210805T011657.0Z,0,0,10,0.000340,unvalidated,Tsukuba-NIES,Japan,36.0513,140.121,45.0,2021,08,05,01,16,2021-08-05 01:16:00,2021-08-05 10:16:00,2021-08-05 01:00:00
2,20210805T012045.7Z,0,0,10,0.000342,unvalidated,Tsukuba-NIES,Japan,36.0513,140.121,45.0,2021,08,05,01,20,2021-08-05 01:20:00,2021-08-05 10:20:00,2021-08-05 01:00:00
3,20210805T012348.6Z,0,0,10,0.000339,unvalidated,Tsukuba-NIES,Japan,36.0513,140.121,45.0,2021,08,05,01,23,2021-08-05 01:23:00,2021-08-05 10:23:00,2021-08-05 01:00:00
4,20210805T014241.8Z,0,0,10,0.000411,unvalidated,Tsukuba-NIES,Japan,36.0513,140.121,45.0,2021,08,05,01,42,2021-08-05 01:42:00,2021-08-05 10:42:00,2021-08-05 02:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124804,20231229T044518.0Z,10,10,10,0.000147,unvalidated,Seoul-SNU,"Korea, Republic of",37.4580,126.951,116.0,2023,12,29,04,45,2023-12-29 04:45:00,2023-12-29 12:45:00,2023-12-29 05:00:00
124805,20231229T044953.8Z,10,10,10,0.000128,unvalidated,Seoul-SNU,"Korea, Republic of",37.4580,126.951,116.0,2023,12,29,04,49,2023-12-29 04:49:00,2023-12-29 12:49:00,2023-12-29 05:00:00
124806,20231229T045711.7Z,10,10,10,0.000131,unvalidated,Seoul-SNU,"Korea, Republic of",37.4580,126.951,116.0,2023,12,29,04,57,2023-12-29 04:57:00,2023-12-29 12:57:00,2023-12-29 05:00:00
124808,20231229T051533.1Z,10,10,10,0.000120,unvalidated,Seoul-SNU,"Korea, Republic of",37.4580,126.951,116.0,2023,12,29,05,15,2023-12-29 05:15:00,2023-12-29 13:15:00,2023-12-29 05:00:00


In [9]:
pandora2['L2HCHOQA'].value_counts()

10    509174
11       198
Name: L2HCHOQA, dtype: int64

In [10]:
# We only use data that is close to GEMS observation time
pandora2['Minute'] = pandora2['Minute'].astype(int)
pandora3 = pandora2[(pandora2['Minute']<=15)|(pandora2['Minute']>=45)].copy()
pandora3

,Time,L1QA,L2fitQA,L2HCHOQA,VCD(moles/m2),Status,Location,Country,Lat,Lon,Altitude,Year,Month,Day,Hour,Minute,Datetime,Localtime,Datetime1
0,20210805T011209.5Z,0,0,10,0.000317,unvalidated,Tsukuba-NIES,Japan,36.0513,140.121,45.0,2021,08,05,01,12,2021-08-05 01:12:00,2021-08-05 10:12:00,2021-08-05 01:00:00
5,20210805T014758.0Z,0,0,10,0.000292,unvalidated,Tsukuba-NIES,Japan,36.0513,140.121,45.0,2021,08,05,01,47,2021-08-05 01:47:00,2021-08-05 10:47:00,2021-08-05 02:00:00
6,20210805T015310.4Z,0,0,10,0.000347,unvalidated,Tsukuba-NIES,Japan,36.0513,140.121,45.0,2021,08,05,01,53,2021-08-05 01:53:00,2021-08-05 10:53:00,2021-08-05 02:00:00
8,20210805T020214.6Z,0,0,10,0.000217,unvalidated,Tsukuba-NIES,Japan,36.0513,140.121,45.0,2021,08,05,02,2,2021-08-05 02:02:00,2021-08-05 11:02:00,2021-08-05 02:00:00
10,20210805T021240.3Z,0,0,10,0.000236,unvalidated,Tsukuba-NIES,Japan,36.0513,140.121,45.0,2021,08,05,02,12,2021-08-05 02:12:00,2021-08-05 11:12:00,2021-08-05 02:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124794,20231229T035051.5Z,10,10,10,0.000081,unvalidated,Seoul-SNU,"Korea, Republic of",37.4580,126.951,116.0,2023,12,29,03,50,2023-12-29 03:50:00,2023-12-29 11:50:00,2023-12-29 04:00:00
124804,20231229T044518.0Z,10,10,10,0.000147,unvalidated,Seoul-SNU,"Korea, Republic of",37.4580,126.951,116.0,2023,12,29,04,45,2023-12-29 04:45:00,2023-12-29 12:45:00,2023-12-29 05:00:00
124805,20231229T044953.8Z,10,10,10,0.000128,unvalidated,Seoul-SNU,"Korea, Republic of",37.4580,126.951,116.0,2023,12,29,04,49,2023-12-29 04:49:00,2023-12-29 12:49:00,2023-12-29 05:00:00
124806,20231229T045711.7Z,10,10,10,0.000131,unvalidated,Seoul-SNU,"Korea, Republic of",37.4580,126.951,116.0,2023,12,29,04,57,2023-12-29 04:57:00,2023-12-29 12:57:00,2023-12-29 05:00:00


In [11]:
pandora3 = pandora3[[  'VCD(moles/m2)',  'Location', 
       'Lat', 'Lon', 'Datetime1']]
pandora3['Year'] = pandora3['Datetime1'].dt.year
pandora3['Month'] = pandora3['Datetime1'].dt.month
pandora3['Day'] = pandora3['Datetime1'].dt.day
pandora3['Hour'] = pandora3['Datetime1'].dt.hour

pandora3['time']= pandora3['Hour']+pandora3['Day']*100+pandora3['Month']*10000+pandora3['Year']*1000000
pandora3['time']= pandora3['time'].astype(int)
pandora3['time']= pandora3['time'].astype(str)

pandora3['ID'] = pandora3['time']+' '+pandora3['Location']

pandora3

/var/folders/4y/knznw84s4p5d2d77tb6wyq6h0000gn/T/ipykernel_72700/2755706391.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pandora3['Year'] = pandora3['Datetime1'].dt.year
/var/folders/4y/knznw84s4p5d2d77tb6wyq6h0000gn/T/ipykernel_72700/2755706391.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pandora3['Month'] = pandora3['Datetime1'].dt.month
/var/folders/4y/knznw84s4p5d2d77tb6wyq6h0000gn/T/ipykernel_72700/2755706391.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

,VCD(moles/m2),Location,Lat,Lon,Datetime1,Year,Month,Day,Hour,time,ID
0,0.000317,Tsukuba-NIES,36.0513,140.121,2021-08-05 01:00:00,2021,8,5,1,2021080501,2021080501 Tsukuba-NIES
5,0.000292,Tsukuba-NIES,36.0513,140.121,2021-08-05 02:00:00,2021,8,5,2,2021080502,2021080502 Tsukuba-NIES
6,0.000347,Tsukuba-NIES,36.0513,140.121,2021-08-05 02:00:00,2021,8,5,2,2021080502,2021080502 Tsukuba-NIES
8,0.000217,Tsukuba-NIES,36.0513,140.121,2021-08-05 02:00:00,2021,8,5,2,2021080502,2021080502 Tsukuba-NIES
10,0.000236,Tsukuba-NIES,36.0513,140.121,2021-08-05 02:00:00,2021,8,5,2,2021080502,2021080502 Tsukuba-NIES
...,...,...,...,...,...,...,...,...,...,...,...
124794,0.000081,Seoul-SNU,37.4580,126.951,2023-12-29 04:00:00,2023,12,29,4,2023122904,2023122904 Seoul-SNU
124804,0.000147,Seoul-SNU,37.4580,126.951,2023-12-29 05:00:00,2023,12,29,5,2023122905,2023122905 Seoul-SNU
124805,0.000128,Seoul-SNU,37.4580,126.951,2023-12-29 05:00:00,2023,12,29,5,2023122905,2023122905 Seoul-SNU
124806,0.000131,Seoul-SNU,37.4580,126.951,2023-12-29 05:00:00,2023,12,29,5,2023122905,2023122905 Seoul-SNU


In [12]:
pandora4 = pandora3.groupby(['ID']).median()
# pandora4['time']= pandora4['Hour']+pandora4['Day']*100+pandora4['Month']*10000+pandora4['Year']*1000000
# pandora4['time']= pandora4['time'].astype(int)
# pandora4['time']= pandora4['time'].astype(str)
pandora4 = pandora4.reset_index()
pandora4[['time', 'loca']] = pandora4['ID'].str.split(' ', n=1, expand=True)

datetimeLis=[]
for j in pandora4['time']:

        datetime_obj = datetime.datetime(int(j[:4]), int(j[4:6]), int(j[6:8]), int(j[8:10]))
        datetime_obj = datetime_obj - timedelta(minutes=15)
        datetimeLis.append(datetime_obj)
pandora4['datetime']=datetimeLis
pandora4['Year'] = pandora4['datetime'].dt.year
pandora4['Month'] = pandora4['datetime'].dt.month
pandora4['Day'] = pandora4['datetime'].dt.day
pandora4['Hour'] = pandora4['datetime'].dt.hour
pandora4['time']= pandora4['Hour']+pandora4['Day']*100+pandora4['Month']*10000+pandora4['Year']*1000000

pandora4 = pandora4[[ 'VCD(moles/m2)', 'Lat', 'Lon', 'Hour',
       'time', 'loca', 'datetime']]
pandora4 = pandora4[pandora4['Hour'].isin([0,1,2,3,4,5,6,7,22,23])]
pandora4

/var/folders/4y/knznw84s4p5d2d77tb6wyq6h0000gn/T/ipykernel_72700/3702788574.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  pandora4 = pandora3.groupby(['ID']).median()


,VCD(moles/m2),Lat,Lon,Hour,time,loca,datetime
0,0.000040,36.7769,126.4938,1,2021010101,Seosan,2021-01-01 01:45:00
1,0.000047,37.4580,126.9510,1,2021010101,Seoul-SNU,2021-01-01 01:45:00
2,0.000039,36.7769,126.4938,2,2021010102,Seosan,2021-01-01 02:45:00
3,0.000080,37.4580,126.9510,2,2021010102,Seoul-SNU,2021-01-01 02:45:00
4,0.000035,36.7769,126.4938,3,2021010103,Seosan,2021-01-01 03:45:00
...,...,...,...,...,...,...,...
72460,0.000050,36.0513,140.1210,22,2023123122,Tsukuba-NIES,2023-12-31 22:45:00
72461,0.000030,35.6200,139.3834,23,2023123123,Tokyo-TMU,2023-12-31 23:45:00
72462,0.000039,36.0661,140.1244,23,2023123123,Tsukuba,2023-12-31 23:45:00
72463,0.000036,36.0513,140.1210,23,2023123123,Tsukuba-NIES,2023-12-31 23:45:00


In [13]:
pandora4 = pandora4.sort_values('time')

In [14]:
pandora4.to_pickle('Pandora_fuh5_MD2')